In [3]:
#Setup Dependencies 
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import requests
import pymongo
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
from webdriver_manager.chrome import ChromeDriverManager

In [4]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [C:\Users\emily\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache




In [5]:
# Begin scraping
url = 'https://mars.nasa.gov/news/'

# Reach out to site
browser.visit(url)

html = browser.html

news_data = BeautifulSoup(html, 'html.parser')

In [9]:
#Find title
news_title = news_data.find_all('div', class_ = 'content_title')[1].text

news_paragraph = news_data.find_all('div', class_ = 'article_teaser_body')[0].text


In [10]:
print(news_title)
print(news_paragraph)

NASA Ingenuity Mars Helicopter Prepares for First Flight
Now uncocooned from its protective carbon-fiber shield, the helicopter is being readied for its next steps.  


In [16]:
#Head to the other site to get the space images 
nasa_url = 'https://www.jpl.nasa.gov'
next_url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'

browser.visit(next_url)

html = browswer.html

image = BeautifulSoup(html, 'html.parser')


WebDriverException: Message: chrome not reachable
  (Session info: chrome=89.0.4389.90)


In [ ]:
image_url = image.find_all('img')[3]["src"]

featured_image_url = nasa_url + image_url

print(featured_image_url)